In [1]:
import json
from datetime import datetime
from pprint import pprint
import numpy as np
import pandas as pd

import os
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from tqdm.notebook import tqdm

In [2]:
from itertools import product


datasets = {}
for name in ('airline', 'hotel', 'heart'):
# for name in ('heart',):
    dataset = {}
    for part, split, typ in product(('X', 'y'), ('train', 'test'), ('bin_', '')):
        dataset[f'{typ}{part}_{split}'] = pd.read_csv(f'./data_preprocessed/{name}_{typ}{part}_{split}.csv')
    datasets[name] = dataset

# datasets['airline']['categorical'] = [
#     'Gender',
#     'Customer Type',
#     # 'Type of Travel',
#     # 'Class',
# ]

# datasets['hotel']['categorical'] = [
#     'deposit_type',
#     'customer_type',
#     # 'arrival_date_year',
#     'market_segment',
#     # 'arrival_date_month',
#     # 'arrival_date_day_of_month',
#     'meal',
#     'hotel'
# ]

# datasets['heart']['categorical'] = [
#     'Diabetes',
# ]

NameError: name 'load_heart' is not defined

In [ ]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lazy_fca_sklearn_wrapper import MyBinarizedBinaryClassifier, MyPatternBinaryClassifier

models_and_param_grids = {
    'naive_bayes': (BernoulliNB(binarize=False), {
        'alpha': [1.0,]
    }),
    'xgboost': (XGBClassifier(random_state=0, objective='binary:logistic'), {
        'n_estimators': [100, 200],
    }),
    'random_forest': (RandomForestClassifier(random_state=0), {
        'n_estimators': [100, 200],
    }),
    'logreg': (LogisticRegression(random_state=0), {
        'C': [0.1, 1], 
        'class_weight': [None, 'balanced']
    }),
    'knn': (KNeighborsClassifier(), {
        'n_neighbors': [5, 10],
        'weights': ['distance'],
    }),
    'catboost': (CatBoostClassifier(verbose=0, random_state=0), {
        'n_estimators': [100, 200],
    }),
    'lazy_fca': (MyBinarizedBinaryClassifier(), {
        'alpha': [0, 0.1, 0.5, 0.9, 1.0],
        'method': ['standard', 'standard-support', 'ratio-support'],
    }),
    'lazy_fca_pat_structures': (MyPatternBinaryClassifier(), {
        # 'alpha': [0, 0.1, 0.5, 0.9, 1.0],
        'alpha': [0.5, 0.9],
        'method': ['standard', 'standard-support', 'ratio-support'],
        # 'method': ['standard', 'standard-support', 'ratio-support'],
    })
}

In [ ]:
from sklearn.pipeline import FunctionTransformer


results = []
experiment_dir = f'./output/{datetime.now()}'
os.makedirs(experiment_dir)

for dataset_name, dataset in tqdm(datasets.items()):
    for model_name, (model, param_grid) in tqdm(models_and_param_grids.items()):
        print(dataset_name, model_name)

        if model_name == 'lazy_fca':
            X_train = dataset['bin_X_train'].astype(bool)
            X_test = dataset['bin_X_test'].astype(bool)
            y_train = dataset['bin_y_train'].astype(bool)
            y_test = dataset['bin_y_test'].astype(bool)
        elif model_name == 'lazy_fca_pat_structures':
            X_train = dataset['X_train']
            model.categorical = [X_train.columns.get_loc(col) for col in  X_train.select_dtypes(bool).columns]
            X_test = dataset['X_test']
            y_train = dataset['bin_y_train'].astype(bool)
            y_test = dataset['bin_y_test'].astype(bool)
            print(model.categorical)
        else:
            X_train = dataset['bin_X_train'].astype(float)
            X_test = dataset['bin_X_test'].astype(float)
            y_train = dataset['bin_y_train'].astype(float)
            y_test = dataset['bin_y_test'].astype(float)

        X_train = np.asarray(X_train)
        X_test = np.asarray(X_test)
        y_test = np.asarray(y_test).ravel()
        y_train = np.asarray(y_train).ravel()
            
        grid_search = GridSearchCV(
            estimator=model,
            cv=StratifiedKFold(n_splits=5),
            param_grid=param_grid,
            n_jobs=3,
            verbose=1,
            scoring=['accuracy', 'f1', 'precision', 'recall'],
            refit='f1',
            error_score='raise'
        )
            
        grid_search.fit(X=X_train, y=y_train)
        y_pred = grid_search.predict(X_test)

        res = {
            'dataset': dataset_name,
            'model': model_name,
            'f1': f1_score(y_test, y_pred),
            'accuracy': accuracy_score(y_test, y_pred),
            'params': grid_search.best_params_
        }
        display(pd.DataFrame([res]))
        with open(f'{experiment_dir}/{model_name}_{dataset_name}.json', 'wt') as f:
            json.dump(res, f)
        results.append(res)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

airline naive_bayes
Fitting 5 folds for each of 1 candidates, totalling 5 fits


,dataset,model,f1,accuracy,params
0,airline,naive_bayes,0.745098,0.74,{'alpha': 1.0}


airline xgboost
Fitting 5 folds for each of 2 candidates, totalling 10 fits


...

/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,dataset,model,f1,accuracy,params
0,heart,lazy_fca,0.311111,0.69,"{'alpha': 0.1, 'method': 'standard'}"


heart lazy_fca_pat_structures
[7, 8, 9]
Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/alex/python_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/python_venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: Un

,dataset,model,f1,accuracy,params
0,heart,lazy_fca_pat_structures,0.311111,0.69,"{'alpha': 0.5, 'method': 'ratio-support'}"


In [ ]:
df_results = pd.DataFrame(results).pivot(
    index='model',
    columns='dataset',
    values=['f1', 'accuracy'],
).swaplevel(axis=1).sort_index(axis=1)
df_results.style.highlight_max(color='red').format(precision=3)

In [ ]:
df_results.style.highlight_max(props='textbf:--rwrap;').format(precision=3).to_latex('table.tex')